<h1>First prototype: encode-decode-predict the immediate nearest neighbor</h1>
In order to check if it's possible to learn a metric minimizer.
Based on https://medium.com/the-modern-scientist/graph-neural-networks-series-part-3-node-embedding-36613cc967d5
and https://machinelearningmastery.com/building-a-binary-classification-model-in-pytorch/ and suggestions of Florian Racoussier.

<h2>1-NN data</h2>
Create a simple dataset for testing the prototype. Each node is connected to at most 15 neighbors in order to provide structure.

In [1]:
from math import sqrt
import torch
from torch_geometric.data import Data
from random import randint
from sys import float_info
from torch_geometric.nn import knn_graph

instances = {}
for k in range(0, 1000):
    nodes = {}
    for i in range(0, 50):
        lat_i = randint(0, 100)
        lon_i = randint(0, 100)
        node_i = (lat_i, lon_i)
        lat_j = randint(0, 100)
        lon_j = randint(0, 100)
        node_j = (lat_j, lon_j)
        nodes[i + 1] = node_i
        nodes[i + 51] = node_j

    dist = {}
    pairs = {}
    for i in range(1, 101):
        for j in range(1, 101):
            if i != j:
                dist[i,j] = sqrt( (nodes[i][0] - nodes[j][0])**2 + (nodes[i][1] - nodes[j][1])**2 )
            else:
                dist[i,j] = float_info.max
    for i in range(1, 101):
        for j in range(1, 101):
            if i not in pairs:
                pairs[i] = j
            if i != j:
                if dist[i,j] < dist[i,pairs[i]]:
                    pairs[i] = j

    nodes[0] = (0,0)
    for i in range(1,101):
        dist[0,i] = sqrt( (nodes[0][0] - nodes[i][0])**2 + (nodes[0][1] - nodes[i][1])**2 )
        dist[i,0] = dist[0,i]
    y = [[0 for _ in range(101)] for _ in range(101)]
    for i in range(101):
        if i > 0:
            y[i][pairs[i]] = 1
                
    instances[k] = {"nodes": nodes, "dist": dist, "y": y}
data_list = []
for instance_name in instances:
    y = torch.tensor(instances[instance_name]["y"], dtype=torch.float)
    x = torch.tensor([instances[instance_name]["nodes"][i] for i in range(0, 101)], dtype=torch.float)
    pos = []
    for i in range(101):
        pos.append(instances[instance_name]["nodes"][i])
    pos = torch.tensor(pos, dtype=torch.double)
    data_list.append(Data(x=x, y=y, edge_index = knn_graph(x, 15), pos=pos))

<h2>1-NN Batching and dividing data into train-test-validation</h2>

In [2]:
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T

def add_edge_labels(graph):
    transform = T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False)
    return transform(graph)

labeled_graphs = [add_edge_labels(graph) for graph in data_list]

train_size = [g[0] for g in labeled_graphs]
val_size = [g[1] for g in labeled_graphs]
test_size = [g[2] for g in labeled_graphs]

train_loader = DataLoader(train_size, batch_size=20, shuffle=True)
val_loader = DataLoader(val_size, batch_size=20, shuffle=False)
test_loader = DataLoader(test_size, batch_size=20, shuffle=False)

<h2>Encoder-Decoder architecture definition</h2>
Add text here

In [3]:
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

model = Net(data_list[0].num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

<h2>Encoder-Decoder train-test-validate routine</h2>
Add text here

In [4]:
from sklearn.metrics import roc_auc_score

def train(loader):
    model.train()
    total_loss = 0

    for batch in loader:
        optimizer.zero_grad()
        z = model.encode(batch.x, batch.edge_index)

        # We perform a new round of negative sampling for every training epoch:
        neg_edge_index = negative_sampling(
            edge_index=batch.edge_index, num_nodes=batch.num_nodes,
            num_neg_samples=batch.edge_label_index.size(1), method='sparse')

        # Concat positive and negative edge indices.
        edge_label_index = torch.cat(
            [batch.edge_label_index, neg_edge_index],
            dim=-1,
        )
        # Label for positive edges: 1, for negative edges: 0.
        edge_label = torch.cat([
            batch.edge_label,
            batch.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        # Note: The model is trained in a supervised manner using the given
        # `edge_label_index` and `edge_label` targets.
        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


@torch.no_grad()
def test(loader):
    model.eval()
    all_out = []
    all_labels = []

    for batch in loader:
        z = model.encode(batch.x, batch.edge_index)
        out = model.decode(z, batch.edge_label_index).view(-1).sigmoid()
        all_out.append(out.cpu().numpy())
        all_labels.append(batch.edge_label.cpu().numpy())

    all_out = np.concatenate(all_out)
    all_labels = np.concatenate(all_labels)
    return roc_auc_score(all_labels, all_out)

In [5]:
# Train/Test Loop
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train(train_loader)
    val_auc = test(val_loader)
    test_auc = test(test_loader)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

Epoch: 001, Loss: 122.2200, Val: 0.8519, Test: 0.8531
Epoch: 002, Loss: 1.4640, Val: 0.8638, Test: 0.8646
Epoch: 003, Loss: 0.7274, Val: 0.8689, Test: 0.8694
Epoch: 004, Loss: 0.5976, Val: 0.8711, Test: 0.8717
Epoch: 005, Loss: 0.5664, Val: 0.8718, Test: 0.8725
Epoch: 006, Loss: 0.5568, Val: 0.8731, Test: 0.8737
Epoch: 007, Loss: 0.5530, Val: 0.8743, Test: 0.8747
Epoch: 008, Loss: 0.5514, Val: 0.8757, Test: 0.8763
Epoch: 009, Loss: 0.5502, Val: 0.8769, Test: 0.8774
Epoch: 010, Loss: 0.5489, Val: 0.8782, Test: 0.8787
Epoch: 011, Loss: 0.5481, Val: 0.8803, Test: 0.8808
Epoch: 012, Loss: 0.5472, Val: 0.8815, Test: 0.8821
Epoch: 013, Loss: 0.5464, Val: 0.8841, Test: 0.8847
Epoch: 014, Loss: 0.5448, Val: 0.8862, Test: 0.8868
Epoch: 015, Loss: 0.5435, Val: 0.8886, Test: 0.8890
Epoch: 016, Loss: 0.5421, Val: 0.8917, Test: 0.8921
Epoch: 017, Loss: 0.5410, Val: 0.8945, Test: 0.8950
Epoch: 018, Loss: 0.5394, Val: 0.8979, Test: 0.8983
Epoch: 019, Loss: 0.5358, Val: 0.9014, Test: 0.9020
Epoch: 020

<h2>1-NN additional statistics</h2>
Add text here

In [6]:
graph_dict = {}
cnt = 0
for graph in data_list:
    z_raw = model.encode(graph.x, graph.edge_index)
    final_edge_index = model.decode_all(z_raw)
    fei = final_edge_index.tolist()
    edges_pred = {k:[] for k in range(101)}
    edges_pred_inv = {k:[] for k in range(101)}
    for i in range(len(fei[0])):
        edges_pred[fei[0][i]].append(fei[1][i])
        edges_pred_inv[fei[1][i]].append(fei[0][i])
    ts0 = graph.edge_index.tolist()
    edges = {k:[] for k in range(101)}
    edges_inv = {k:[] for k in range(101)}
    for i in range(len(ts0[0])):
        edges[ts0[0][i]].append(ts0[1][i])
        edges_inv[ts0[1][i]].append(ts0[0][i])
    originals = {}
    predictions = {}
    for i in range(101):
        originals[i] = set(edges[i] + edges_inv[i])
        predictions[i] = set(edges_pred[i] + edges_pred_inv[i])
    graph_dict[cnt] = {"real": originals, "preds": predictions}
    cnt += 1

confusion_dict = {}
true_positives = []
false_positives = []
true_negatives = []
false_negatives = []
for key in graph_dict:
    real = graph_dict[key]["real"]
    pred = graph_dict[key]["preds"]
    node_matrix = {}
    for i in range(101):
        tp = len(pred[i].intersection(real[i]))
        fp = len(pred[i] - real[i])
        real_neg = set([j for j in range(101)]) - {i} - real[i]
        pred_neg = set([j for j in range(101)]) - {i} - pred[i]
        tn = len(pred_neg.intersection(real_neg))
        fn = len(pred_neg - real_neg)
        total = tp + fp + tn + fn
        node_matrix[i] = {"tp": tp, "fp": fp, "tn": tn, "fn": fn, "total": total}
        true_positives.append(tp/total)
        false_positives.append(fp/total)
        true_negatives.append(tn/total)
        false_negatives.append(fn/total)
    confusion_dict[key] = node_matrix

In [7]:
from statistics import mean, stdev
print("true positives mean={0:.2f}, stdev={1:.2f}".format(mean(true_positives), stdev(true_positives)))
print("false positives mean={0:.2f}, stdev={1:.2f}".format(mean(false_positives), stdev(false_positives)))
print("true negatives mean={0:.2f}, stdev={1:.2f}".format(mean(true_negatives), stdev(true_negatives)))
print("false negatives mean={0:.2f}, stdev={1:.2f}".format(mean(false_negatives), stdev(false_negatives)))

true positives mean=0.17, stdev=0.03
false positives mean=0.28, stdev=0.10
true negatives mean=0.54, stdev=0.10
false negatives mean=0.00, stdev=0.00


There's some notion of closeness. It over-links, but rarely avoids a close node. Also, a middle stage as this is not always a requirement. We want to encode the data in embeddings that can be used for the next stage. The quality will be measured then over the ability to learn closeness.

<h2>1-NN data preparation for second stage</h2>
Add text here

In [8]:
from random import sample
data_raw_dict = {
    "positives": [],
    "negatives": []
}
cnt = 0
pos_cnt = 0
neg_cnt = 0
for graph in data_list:
    neg_cnt = 0
    ng_matrix = graph.y.tolist()
    encoding_matrix = model.encode(graph.x, graph.edge_index).tolist()
    for i in range(101):
        for j in sample(range(101), 30):
            if i == j:
                # if pos_cnt < 80000:
                #     data_raw_dict["positives"].append(encoding_matrix[i]+encoding_matrix[i])
                #     pos_cnt += 1
                continue
            else:
                if ng_matrix[i][j] > 0.5:
                    if pos_cnt < 80000:
                        data_raw_dict["positives"].append(encoding_matrix[i]+encoding_matrix[j]+[1])
                        pos_cnt += 1
                else:
                    if pos_cnt < 80000 and neg_cnt < 1010:
                        data_raw_dict["negatives"].append(encoding_matrix[i]+encoding_matrix[j]+[0])
                        neg_cnt += 1

In [9]:
pre_tensor = data_raw_dict["positives"][:5000] + data_raw_dict["negatives"][:5000]
main_tensor = torch.tensor(pre_tensor, dtype=torch.float32)

main_tensor = main_tensor[torch.randperm(main_tensor.size()[0])]
labels = main_tensor[:,-1:]
embeddings = main_tensor[:,:-1]

<h2>1-NN Wide or Deep network</h2>
Add text here

In [10]:
import torch.nn as nn

class Wide(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(128, 64)
        self.relu = nn.ReLU()
        self.output = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
 
    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = self.sigmoid(self.output(x))
        return x

class Deep(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(128, 128)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(128, 128)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(128, 128)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

In [11]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
 
def model_train(model, X_train, y_train, X_val, y_val):
    # loss function and optimizer
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
 
    n_epochs = 20   # number of epochs to run
    batch_size = 10  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)
 
    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None
 
    for epoch in range(n_epochs):
        model.train()
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())
    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    return best_acc

In [12]:
from sklearn.model_selection import StratifiedKFold, train_test_split

# train-test split: Hold out the test set for final model evaluation
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, train_size=0.7, shuffle=True)
 
# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True)
cv_scores_wide = []
for train, test in kfold.split(X_train, y_train):
    # create model, train, and get accuracy
    model = Wide()
    acc = model_train(model, X_train[train], y_train[train], X_train[test], y_train[test])
    print("Accuracy (wide): %.2f" % acc)
    cv_scores_wide.append(acc)
cv_scores_deep = []
for train, test in kfold.split(X_train, y_train):
    # create model, train, and get accuracy
    model = Deep()
    acc = model_train(model, X_train[train], y_train[train], X_train[test], y_train[test])
    print("Accuracy (deep): %.2f" % acc)
    cv_scores_deep.append(acc)
 
# evaluate the model
wide_acc = np.mean(cv_scores_wide)
wide_std = np.std(cv_scores_wide)
deep_acc = np.mean(cv_scores_deep)
deep_std = np.std(cv_scores_deep)
print("Wide: %.2f%% (+/- %.2f%%)" % (wide_acc*100, wide_std*100))
print("Deep: %.2f%% (+/- %.2f%%)" % (deep_acc*100, deep_std*100))

Accuracy (wide): 0.93
Accuracy (wide): 0.94
Accuracy (wide): 0.92
Accuracy (wide): 0.94
Accuracy (wide): 0.93
Accuracy (deep): 0.95
Accuracy (deep): 0.94
Accuracy (deep): 0.95
Accuracy (deep): 0.95
Accuracy (deep): 0.95
Wide: 93.19% (+/- 0.53%)
Deep: 94.86% (+/- 0.42%)


In [13]:
# rebuild model with full set of training data
if wide_acc > deep_acc:
    print("Retrain a wide model")
    model = Wide()
else:
    print("Retrain a deep model")
    model = Deep()
acc = model_train(model, X_train, y_train, X_test, y_test)
print(f"Final model accuracy: {acc*100:.2f}%")

Retrain a deep model
Final model accuracy: 95.13%


In [14]:
model.eval()
with torch.no_grad():
    # Test out inference with 5 samples
    for i in range(5):
        y_pred = model(X_test[i:i+1])
        print(f"{X_test[i].numpy()} -> {y_pred[0].numpy()} (expected {y_test[i].numpy()})")

[ 4.1717339e-01 -5.7865076e-02 -1.4151484e-02  1.5557876e-02
 -4.2768332e-01 -4.4370964e-01  3.3552490e-02  4.1792460e-02
 -4.4831714e-01 -7.0630580e-02  2.1677202e-01  2.2268556e-01
 -6.3116723e-01  8.4077612e-02  5.1827902e-01  3.1405866e-01
 -2.0323183e-01  8.0681220e-02  1.3163482e-01 -1.4057782e-01
 -1.0440167e-02  9.3202174e-02  1.8332930e-01  2.2968042e-01
  1.0498860e-01 -2.6336920e-01  5.9151962e-02  5.4599382e-02
 -5.2246571e-02 -2.9523142e-02  2.7236146e-01 -5.9649184e-02
 -2.9510462e-01 -2.3909754e-01 -2.0827483e-01 -4.0251181e-01
 -1.6330114e-01 -2.0440927e-01 -7.0687167e-02  2.9918218e-01
  9.8987930e-03  9.5853359e-02 -2.3753516e-01 -1.8214691e-01
 -4.3035455e-02 -1.5556519e-02 -1.2626988e-01  1.3818063e-01
 -2.8056109e-01  5.3613499e-02 -2.5029457e-01  3.6895238e-02
 -1.3619283e-01  5.3167921e-01 -3.1269984e-03 -3.6275232e-01
  2.5250453e-01  1.8157607e-01 -2.1313316e-01 -8.1577316e-02
  1.5478041e-02  2.6419458e-01  5.4498559e-01  2.4573167e-01
  4.3307829e-01 -5.64706

The one above is an example for visual validation

<h1>NgSets prototype, undirected</h1>
Once validated the simple prototype, let's continue to the actual problem

<h2>Load the data</h2>
Here an "export" folder with .txt instance files used on the CTWVRP project, and a .csv file with the neighborhoods.

In [15]:
import os

data_files_list = ["./export/"+f for f in os.listdir("./export") ]
instance_dict = {}
for dir_str in data_files_list:
    with open(dir_str, 'r') as text_file:
        cnt = 0
        instance = ""
        for line in text_file:
            if cnt < 9:
                if cnt == 0:
                    instance = line.split()[0]
                    instance_dict[instance] = []
                cnt += 1
                continue
            split_line = line.split()
            instance_dict[instance].append([int(i) for i in split_line])
        text_file.close()

ng_dict = {}
cnt = -1
with open("ng_outs.csv", 'r') as text_file:
    for line in text_file:
        if cnt < 2:
            cnt += 1
            continue
        raw_line = line.strip()
        split_line_list = raw_line.split(sep=";")
        instance = split_line_list[3]
        if instance not in ng_dict:
            ng_dict[instance] = [[0 for i in range(101)]]
        ng_dict[instance].append([0] + [int(i) for i in split_line_list[5:-1]])
        if len(split_line_list[5:-1]) != 100:
            print("case found for instance "+instance)
    text_file.close()

In [16]:
from math import sqrt
import torch
import torch_geometric as tg
from torch_geometric.data import Data
import networkx as nx
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, GraphConv, global_add_pool
from torch_geometric.loader import DataLoader

In [17]:
# data_list = []
# for instance_name in ng_dict:
#     y = torch.tensor(ng_dict[instance_name], dtype=torch.double)
#     x = torch.tensor(instance_dict[instance_name], dtype=torch.double)
#     attr = [[i] for i in range(n_edges)]
#     loc_dict = {(i[0],j[0]): sqrt((i[1]-j[1])**2 + (i[2]-j[2])**2) for i in instance_dict[instance_name] for j in instance_dict[instance_name]}
#     cnt = -1
#     for i in range(101):
#         for j in range(101):
#             if i != j:
#                 cnt += 1
#                 attr[cnt].append(loc_dict[i,j])
#     attr = torch.tensor(attr, dtype=torch.double)
#     pos = []
#     for i in instance_dict[instance_name]:
#         pos.append([i[1], i[2]])
#     pos = torch.tensor(pos, dtype=torch.double)
#     data_list.append(Data(x=x, y=y, edge_index=edge_index, pos=pos, edge_attr=attr))

data_list = []
for instance_name in ng_dict:
    y = torch.tensor(ng_dict[instance_name], dtype=torch.float)
    x = torch.tensor(instance_dict[instance_name], dtype=torch.float)
    pos = []
    for i in instance_dict[instance_name]:
        pos.append([i[1], i[2]])
    pos = torch.tensor(pos, dtype=torch.double)
    data_list.append(Data(x=x, y=y, edge_index = knn_graph(x, 15), pos=pos))

<h2>Prepare data as in prototype</h2>
Add text here

In [22]:
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T

def add_edge_labels(graph):
    transform = T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False)
    return transform(graph)

labeled_graphs = [add_edge_labels(graph) for graph in data_list]

train_size = [g[0] for g in labeled_graphs]
val_size = [g[1] for g in labeled_graphs]
test_size = [g[2] for g in labeled_graphs]

train_loader = DataLoader(train_size, batch_size=500, shuffle=True)
val_loader = DataLoader(val_size, batch_size=500, shuffle=False)
test_loader = DataLoader(test_size, batch_size=500, shuffle=False)

<h2>Encode-decode routine</h2>

In [37]:
import numpy as np
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [38]:
model_ed = Net(data_list[0].num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model_ed.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [39]:
from sklearn.metrics import roc_auc_score

def train(loader):
    model_ed.train()
    total_loss = 0

    for batch in loader:
        optimizer.zero_grad()
        z = model_ed.encode(batch.x, batch.edge_index)

        # We perform a new round of negative sampling for every training epoch:
        neg_edge_index = negative_sampling(
            edge_index=batch.edge_index, num_nodes=batch.num_nodes,
            num_neg_samples=batch.edge_label_index.size(1), method='sparse')

        # Concat positive and negative edge indices.
        edge_label_index = torch.cat(
            [batch.edge_label_index, neg_edge_index],
            dim=-1,
        )
        # Label for positive edges: 1, for negative edges: 0.
        edge_label = torch.cat([
            batch.edge_label,
            batch.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        # Note: The model is trained in a supervised manner using the given
        # `edge_label_index` and `edge_label` targets.
        out = model_ed.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


@torch.no_grad()
def test(loader):
    model_ed.eval()
    all_out = []
    all_labels = []

    for batch in loader:
        z = model_ed.encode(batch.x, batch.edge_index)
        out = model_ed.decode(z, batch.edge_label_index).view(-1).sigmoid()
        all_out.append(out.cpu().numpy())
        all_labels.append(batch.edge_label.cpu().numpy())

    all_out = np.concatenate(all_out)
    all_labels = np.concatenate(all_labels)
    return roc_auc_score(all_labels, all_out)

In [40]:
# Train/Test Loop
best_val_auc = final_test_auc = 0
for epoch in range(1, 21):
    loss = train(train_loader)
    val_auc = test(val_loader)
    test_auc = test(test_loader)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

Epoch: 001, Loss: 9919.4426, Val: 0.5599, Test: 0.5596
Epoch: 002, Loss: 68.1077, Val: 0.6494, Test: 0.6487
Epoch: 003, Loss: 18.2747, Val: 0.6678, Test: 0.6669
Epoch: 004, Loss: 11.6792, Val: 0.6743, Test: 0.6737
Epoch: 005, Loss: 8.9833, Val: 0.6787, Test: 0.6783
Epoch: 006, Loss: 7.3602, Val: 0.6872, Test: 0.6865
Epoch: 007, Loss: 6.2161, Val: 0.6883, Test: 0.6877
Epoch: 008, Loss: 5.3317, Val: 0.7015, Test: 0.7010
Epoch: 009, Loss: 4.5943, Val: 0.7094, Test: 0.7091
Epoch: 010, Loss: 3.9720, Val: 0.7196, Test: 0.7193
Epoch: 011, Loss: 3.4475, Val: 0.7297, Test: 0.7295
Epoch: 012, Loss: 2.9939, Val: 0.7362, Test: 0.7365
Epoch: 013, Loss: 2.6055, Val: 0.7397, Test: 0.7401
Epoch: 014, Loss: 2.2750, Val: 0.7417, Test: 0.7422
Epoch: 015, Loss: 1.9978, Val: 0.7410, Test: 0.7415
Epoch: 016, Loss: 1.7775, Val: 0.7392, Test: 0.7399
Epoch: 017, Loss: 1.5995, Val: 0.7371, Test: 0.7377
Epoch: 018, Loss: 1.4543, Val: 0.7332, Test: 0.7340
Epoch: 019, Loss: 1.3433, Val: 0.7337, Test: 0.7346
Epoch:

<h2>Check on values</h2>

In [28]:
graph_dict = {}
cnt = 0
for graph in data_list:
    z_raw = model_ed.encode(graph.x, graph.edge_index)
    final_edge_index = model_ed.decode_all(z_raw)
    fei = final_edge_index.tolist()
    edges_pred = {k:[] for k in range(101)}
    edges_pred_inv = {k:[] for k in range(101)}
    for i in range(len(fei[0])):
        edges_pred[fei[0][i]].append(fei[1][i])
        edges_pred_inv[fei[1][i]].append(fei[0][i])
    ts0 = graph.edge_index.tolist()
    edges = {k:[] for k in range(101)}
    edges_inv = {k:[] for k in range(101)}
    for i in range(len(ts0[0])):
        edges[ts0[0][i]].append(ts0[1][i])
        edges_inv[ts0[1][i]].append(ts0[0][i])
    originals = {}
    predictions = {}
    for i in range(101):
        originals[i] = set(edges[i] + edges_inv[i])
        predictions[i] = set(edges_pred[i] + edges_pred_inv[i])
    graph_dict[cnt] = {"real": originals, "preds": predictions}
    cnt += 1

confusion_dict = {}
true_positives = []
false_positives = []
true_negatives = []
false_negatives = []
for key in graph_dict:
    real = graph_dict[key]["real"]
    pred = graph_dict[key]["preds"]
    node_matrix = {}
    for i in range(101):
        tp = len(pred[i].intersection(real[i]))
        fp = len(pred[i] - real[i])
        real_neg = set([j for j in range(101)]) - {i} - real[i]
        pred_neg = set([j for j in range(101)]) - {i} - pred[i]
        tn = len(pred_neg.intersection(real_neg))
        fn = len(pred_neg - real_neg)
        total = tp + fp + tn + fn
        node_matrix[i] = {"tp": tp, "fp": fp, "tn": tn, "fn": fn, "total": total}
        true_positives.append(tp/total)
        false_positives.append(fp/total)
        true_negatives.append(tn/total)
        false_negatives.append(fn/total)
    confusion_dict[key] = node_matrix

In [29]:
from statistics import mean, stdev
print("true positives mean={0:.2f}, stdev={1:.2f}".format(mean(true_positives), stdev(true_positives)))
print("false positives mean={0:.2f}, stdev={1:.2f}".format(mean(false_positives), stdev(false_positives)))
print("true negatives mean={0:.2f}, stdev={1:.2f}".format(mean(true_negatives), stdev(true_negatives)))
print("false negatives mean={0:.2f}, stdev={1:.2f}".format(mean(false_negatives), stdev(false_negatives)))

true positives mean=0.17, stdev=0.03
false positives mean=0.65, stdev=0.18
true negatives mean=0.18, stdev=0.18
false negatives mean=0.00, stdev=0.00


<h2>NgLearning</h2>

In [30]:
from random import sample
data_raw_dict = {
    "positives": [],
    "negatives": []
}
cnt = 0
pos_cnt = 0
neg_cnt = 0
for graph in data_list:
    neg_cnt = 0
    ng_matrix = graph.y.tolist()
    encoding_matrix = model.encode(graph.x, graph.edge_index).tolist()
    for i in range(101):
        for j in sample(range(101), 30):
            if i == j:
                # if pos_cnt < 80000:
                #     data_raw_dict["positives"].append(encoding_matrix[i]+encoding_matrix[i])
                #     pos_cnt += 1
                continue
            else:
                if ng_matrix[i][j] > 0.5:
                    if pos_cnt < 80000:
                        data_raw_dict["positives"].append(encoding_matrix[i]+encoding_matrix[j]+[1])
                        pos_cnt += 1
                else:
                    if pos_cnt < 80000 and neg_cnt < 1010:
                        data_raw_dict["negatives"].append(encoding_matrix[i]+encoding_matrix[j]+[0])
                        neg_cnt += 1

In [31]:
pre_tensor = data_raw_dict["positives"][:5000] + data_raw_dict["negatives"][:5000]
main_tensor = torch.tensor(pre_tensor, dtype=torch.float32)

main_tensor = main_tensor[torch.randperm(main_tensor.size()[0])]
labels = main_tensor[:,-1:]
embeddings = main_tensor[:,:-1]

<h2>Learning stage</h2>

In [32]:
class Deep(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(128, 128)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(128, 128)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(128, 128)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

In [33]:
def model_train(model, X_train, y_train, X_val, y_val):
    # loss function and optimizer
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
 
    n_epochs = 20   # number of epochs to run
    batch_size = 10  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)
 
    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None
 
    for epoch in range(n_epochs):
        model.train()
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())
    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    return best_acc

In [41]:
from sklearn.model_selection import StratifiedKFold, train_test_split

# train-test split: Hold out the test set for final model evaluation
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, train_size=0.7, shuffle=True)
 
# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True)
cv_scores_deep = []
for train, test in kfold.split(X_train, y_train):
    # create model, train, and get accuracy
    model_ls = Deep()
    acc = model_train(model_ls, X_train[train], y_train[train], X_train[test], y_train[test])
    print("Accuracy (deep): %.2f" % acc)
    cv_scores_deep.append(acc)
 
# evaluate the model
deep_acc = np.mean(cv_scores_deep)
deep_std = np.std(cv_scores_deep)
print("Deep: %.2f%% (+/- %.2f%%)" % (deep_acc*100, deep_std*100))

Accuracy (deep): 0.97
Accuracy (deep): 0.97
Accuracy (deep): 0.96
Accuracy (deep): 0.97
Accuracy (deep): 0.96
Deep: 96.46% (+/- 0.45%)


In [42]:
model_ls = Deep()
acc = model_train(model_ls, X_train, y_train, X_test, y_test)
print(f"Final model accuracy: {acc*100:.2f}%")

Final model accuracy: 97.37%


In [43]:
model_ls.eval()
with torch.no_grad():
    # Test out inference with 5 samples
    for i in range(5):
        y_pred = model_ls(X_test[i:i+1])
        print(f"{X_test[i].numpy()} -> {y_pred[0].numpy()} (expected {y_test[i].numpy()})")

[ 0.46387836  0.1943435  -0.37253988  0.21790805  0.29583716  0.45350713
 -0.44931048  0.97286505  0.15320109  0.05452149 -0.63882375 -1.2315071
  0.15864943  0.7793971   0.8734473   0.8504205   1.2025108  -0.08820149
  1.1138158  -0.31058282 -0.39535397 -0.41618708 -0.4199708   0.3125999
 -0.00468891  0.03943263 -0.5514713   0.3135659  -0.6818434  -0.28593165
  0.64737177 -1.148654   -0.2523658   0.36917987 -0.08944561  0.95397073
  0.39610565  0.9281209  -0.06270041  0.11733012  0.49224165  0.11228694
  0.39218405 -0.42325312  0.306798   -0.7165122  -0.7366651   1.0995601
 -1.1010973  -0.04442366  0.06648539 -0.4796762  -0.24985504  0.7891465
 -0.09640184  0.2128881  -0.41142714 -0.978595    0.54114723  0.07069811
 -0.79138887 -0.12155458  0.4828937   0.263994    0.3635391   0.1962971
 -0.36730343  0.17375365  0.2576986   0.45053726 -0.39176714  0.8915362
  0.14990321  0.01941784 -0.6330897  -1.1591773   0.16111828  0.7265636
  0.83141446  0.7369272   1.1015825  -0.05950847  1.075030

Test on instance 428

In [48]:
z_raw = model_ed.encode(data_list[428].x, data_list[428].edge_index)
preds = {}
for i in range(len(z_raw)):
    for j in range(len(z_raw)):
        if i != j:
            node_i = z_raw[i].tolist()
            node_j = z_raw[j].tolist()
            target = data_list[428].y[i][j]
            preds[i,j] = {"pred":model_ls(torch.tensor(node_i+node_j, dtype=torch.float)),"target":target}

In [60]:
preds[95,85]

{'pred': tensor([0.0354], grad_fn=<SigmoidBackward0>), 'target': tensor(0.)}

Still space for improvement. I'd consider more data, undirected case for encoding and add tw into the structure of the graph